# SSFM Hypno Building

## Building a Toy Hypnogram

First will get the logic working with a toy example:

In [4]:
import sleep_score_for_me.v1.ssfm_v1 as ssfm
import pandas as pd
import numpy as np

In [5]:
#Makes our toy dataframe:

dti = pd.date_range("2020-01-01T09:00:12.00", periods=100, freq="4S")
ex = pd.Series(np.random.randn(len(dti)), index=dti)
ex = ex.to_frame(name='param1')
ex['param2'] = np.random.randn(len(dti))
ex['state'] = np.nan

In [6]:
# This first assigns NREM based on a param1 threshold:
ex.loc[ex.param1 >= 0, 'state'] = 'NREM'

# This then assigns wake vs rem based on param2:
ex.loc[np.logical_and(ex.param2 >=0, ex.state != 'NREM'), 'state'] = 'Wake'
ex.loc[np.logical_and(ex.param2 <0, ex.state != 'NREM'), 'state'] = 'REM'

- Now to build the hypnogram

In [7]:
# First let's try a boolean series to get the NREM epocs:
s1 = ex['state']
bool1 = s1 == 'NREM'
bool2 = s1 == 'Wake'
bool3 = s1 == 'REM'

In [8]:
bool1

2020-01-01 09:00:12    False
2020-01-01 09:00:16    False
2020-01-01 09:00:20     True
2020-01-01 09:00:24     True
2020-01-01 09:00:28     True
                       ...  
2020-01-01 09:06:32    False
2020-01-01 09:06:36     True
2020-01-01 09:06:40    False
2020-01-01 09:06:44    False
2020-01-01 09:06:48    False
Freq: 4S, Name: state, Length: 100, dtype: bool

In [9]:
# Then we can segment that boolean series: 
nrem_tups = ssfm.starts_and_ends(bool1)
wake_tups = ssfm.starts_and_ends(bool2)
rem_tups = ssfm.starts_and_ends(bool3)

passing indexing error
passing indexing error
passing indexing error


In [10]:
# Then we make each boolean series into a partial-hypnogram:

nrem_hyp = pd.DataFrame(columns = ['state', 'end_time', 'start_time', 'duration'])
nrem_hyp[['start_time', 'end_time']] = nrem_tups
nrem_hyp['duration'] = nrem_hyp.end_time - nrem_hyp.start_time
nrem_hyp['state'] = 'NREM'

wake_hyp = pd.DataFrame(columns = ['state', 'end_time', 'start_time', 'duration'])
wake_hyp[['start_time', 'end_time']] = wake_tups
wake_hyp['duration'] = wake_hyp.end_time - wake_hyp.start_time
wake_hyp['state'] = 'Wake'

rem_hyp = pd.DataFrame(columns = ['state', 'end_time', 'start_time', 'duration'])
rem_hyp[['start_time', 'end_time']] = rem_tups
rem_hyp['duration'] = rem_hyp.end_time - rem_hyp.start_time
rem_hyp['state'] = 'REM'

In [11]:
# Then we concatenate and sort:

toy_hypno = pd.concat([nrem_hyp, wake_hyp, rem_hyp])
toy_hypno = toy_hypno.sort_values('start_time')

In [12]:
# And plot it:
ax = plot_hypnogram_overlay(toy_hypno)

NameError: name 'plot_hypnogram_overlay' is not defined

In [ ]:
def starts_and_ends(s, minimum_duration=np.timedelta64(3, 's')):
    start_times = np.empty(0)
    end_times = np.empty(0)
    period = s.index[1] - s.index[0]

    s_trues = s[s==True]
    ix = s_trues.index
    ix_counter = np.arange(0, len(ix))

    try:
        for i in ix_counter:
            if (ix[i] - period) != ix[i-1]:
                start_times = np.append(start_times, ix[i])
            if (ix[i] + period) != ix[i+1]:
                end_times = np.append(end_times, (ix[i] + period))
            elif np.logical_and((ix[i] + period) == ix[i+1], (ix[i] - period) == ix[i-1]):
                pass 
    except IndexError:
        print('passing indexing error')
        pass
    end_times = np.append(end_times, (ix[ix_counter.max()] + period))
    return [(start_time, end_time)
            for start_time, end_time in zip(start_times, end_times)
            if end_time >= (start_time + minimum_duration)]